In [0]:
%scala
import org.apache.spark.ml.recommendation.ALS
import org.apache.spark.sql.functions._
import org.apache.spark.ml.tuning.{ParamGridBuilder,TrainValidationSplit}
import org.apache.spark.ml.evaluation.RegressionEvaluator

import org.apache.spark.ml.recommendation.ALS
import org.apache.spark.sql.functions._
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.ml.evaluation.RegressionEvaluator

In [0]:
dbutils.fs.ls("FileStore/tables/")

Out[1]: [FileInfo(path='dbfs:/FileStore/tables/29103444_bank.csv', name='29103444_bank.csv', size=5834917, modificationTime=1717028119000),
 FileInfo(path='dbfs:/FileStore/tables/Customer_Gender.csv', name='Customer_Gender.csv', size=7628, modificationTime=1717027822000),
 FileInfo(path='dbfs:/FileStore/tables/movie_ratings-1.csv', name='movie_ratings-1.csv', size=1373062, modificationTime=1723149511000),
 FileInfo(path='dbfs:/FileStore/tables/movie_ratings-2.csv', name='movie_ratings-2.csv', size=1373062, modificationTime=1723149764000),
 FileInfo(path='dbfs:/FileStore/tables/movie_ratings.csv', name='movie_ratings.csv', size=1373062, modificationTime=1723149478000)]

In [0]:
%scala
val df = spark.read.option("header","true").option("inferSchema","true").format("csv").load("dbfs:/FileStore/tables/movie_ratings.csv")
df.printSchema()

root
-- userId: integer (nullable = true)
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)

df: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 1 more field]

In [0]:
%scala
df.count()

res2: Long = 100004

In [0]:
%scala
df.head(10)

res3: Array[org.apache.spark.sql.Row] = Array([1,31,2.5], [1,1029,3.0], [1,1061,3.0], [1,1129,2.0], [1,1172,4.0], [1,1263,2.0], [1,1287,2.0], [1,1293,2.0], [1,1339,3.5], [1,1343,2.0])

In [0]:
%scala
val Array(train,test) = df.randomSplit(Array(0.75,0.25),seed=81)

train: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: int, movieId: int ... 1 more field]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: int, movieId: int ... 1 more field]

In [0]:
%scala
train.count()

res5: Long = 74958

In [0]:
%scala
test.count()

res6: Long = 25046

In [0]:
%scala
val als = new ALS().setMaxIter(10).setRegParam(0.01).setUserCol("userId").setItemCol("movieId").setRatingCol("rating").setColdStartStrategy("drop")

als: org.apache.spark.ml.recommendation.ALS = als_8ecae1645810

In [0]:
%scala
val als_model = als.fit(train)

als_model: org.apache.spark.ml.recommendation.ALSModel = ALSModel: uid=als_8ecae1645810, rank=10

In [0]:
%scala
val predictions = als_model.transform(test)

predictions: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 2 more fields]

In [0]:
%scala
predictions.show()

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 148| 58| 4.0| 4.0812144|
 148| 364| 4.0| 4.270048|
 148| 492| 3.5| 5.2082863|
 148| 904| 5.0| 4.4164224|
 148| 913| 4.0| 4.2206674|
 148| 919| 4.5| 4.8338966|
 148| 1097| 5.0| 4.1097293|
 148| 1252| 5.0| 4.207368|
 148| 1269| 5.0| 3.469981|
 148| 1304| 4.0| 4.5474505|
 148| 1610| 5.0| 3.9185278|
 148| 1690| 3.0| 2.9940963|
 148| 1909| 3.5| 2.871802|
 148| 2053| 2.5| 1.9882214|
 148| 2080| 4.5| 4.1181545|
 148| 2628| 2.5| 3.2070446|
 148| 3256| 4.0| 3.952194|
 148| 3386| 3.5| 4.374929|
 148| 3462| 5.0| 4.214675|
 148| 3671| 4.0| 3.6637778|
+------+-------+------+----------+
only showing top 20 rows

In [0]:
%scala
val errors = predictions.select(abs($"rating"-$"prediction"))
errors.show()

+--------------------------+
abs((rating - prediction))|
+--------------------------+
 0.08121442794799805|
 0.2700481414794922|
 1.7082862854003906|
 0.5835776329040527|
 0.22066736221313477|
 0.3338966369628906|
 0.8902707099914551|
 0.7926321029663086|
 1.5300190448760986|
 0.5474505424499512|
 1.0814721584320068|
 0.005903720855712891|
 0.6281979084014893|
 0.5117785930633545|
 0.38184547424316406|
 0.7070446014404297|
 0.0478060245513916|
 0.8749289512634277|
 0.7853250503540039|
 0.33622217178344727|
+--------------------------+
only showing top 20 rows

errors: org.apache.spark.sql.DataFrame = [abs((rating - prediction)): double]

In [0]:
%scala
errors.na.drop().describe().show()

+-------+--------------------------+
summary|abs((rating - prediction))|
+-------+--------------------------+
 count| 24104|
 mean| 0.8806310943793622|
 stddev| 0.7821798148167962|
 min| 3.385543823242187...|
 max| 8.073600769042969|
+-------+--------------------------+

In [0]:
%scala
val paramGrid = new ParamGridBuilder().addGrid(als.regParam,Array(0.01,0.1,1.0)).build()

val evaluator = new RegressionEvaluator().setMetricName("rmse").setLabelCol("rating").setPredictionCol("prediction")

val trainValidationSplit = new TrainValidationSplit().setEstimator(als).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setTrainRatio(0.8)

val model = trainValidationSplit.fit(train)

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	als_8ecae1645810-regParam: 0.01
}, {
	als_8ecae1645810-regParam: 0.1
}, {
	als_8ecae1645810-regParam: 1.0
})
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_a9ea7a1d6d15, metricName=rmse, throughOrigin=false
trainValidationSplit: org.apache.spark.ml.tuning.TrainValidationSplit = tvs_23b1746297d3
model: org.apache.spark.ml.tuning.TrainValidationSplitModel = TrainValidationSplitModel: uid=tvs_23b1746297d3, bestModel=ALSModel: uid=als_8ecae1645810, rank=10, trainRatio=0.8

In [0]:
%scala 
val predictions = model.transform(test)
predictions.show()

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 148| 58| 4.0| 4.162006|
 148| 364| 4.0| 4.0146947|
 148| 492| 3.5| 4.532341|
 148| 904| 5.0| 4.5658894|
 148| 913| 4.0| 4.518343|
 148| 919| 4.5| 4.4265337|
 148| 1097| 5.0| 3.8710222|
 148| 1252| 5.0| 4.3655357|
 148| 1269| 5.0| 3.9691014|
 148| 1304| 4.0| 4.396582|
 148| 1610| 5.0| 3.8033426|
 148| 1690| 3.0| 2.931346|
 148| 1909| 3.5| 3.3113713|
 148| 2053| 2.5| 2.4967473|
 148| 2080| 4.5| 3.9988854|
 148| 2628| 2.5| 3.0480404|
 148| 3256| 4.0| 3.9032779|
 148| 3386| 3.5| 3.8895802|
 148| 3462| 5.0| 4.432053|
 148| 3671| 4.0| 4.1945276|
+------+-------+------+----------+
only showing top 20 rows

predictions: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 2 more fields]

In [0]:
%scala
val rmse = evaluator.evaluate(predictions)
println("RMSE:",rmse)

(RMSE:,0.9189594041956963)
rmse: Double = 0.9189594041956963

In [0]:
%scala
val r2_eval = new RegressionEvaluator().setLabelCol("rating").setMetricName("r2")
println("R2 Score:",r2_eval.evaluate(predictions))

(R2 Score:,0.2346683710922698)
r2_eval: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_4bc6d59232ca, metricName=r2, throughOrigin=false